# Opik Optimizer (beta)

Welcome to the Opik Opitimizer beta program! In this notebook we'll walk through a basic examples of a number of an optimizer algorithm.

## Setup

First, let's install the needed Python packages.

We'll of course need `opik`:

In [1]:
%pip install opik --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.6 MB/s eta 0:00:00


Next, we'll install the beta version of the `opik-optimizer`:

In [4]:
!pip install git+https://github.com/comet-ml/opik#subdirectory=sdks/opik_optimizer --upgrade --quiet > pip-install.log 2>&1

[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

You can use your own workspace.

In [5]:
import opik

# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "dsblank" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [6]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


To optimize any prompt, we'll need:

1. A starting prompt
2. A metric
3. A dataset

For this initial test, we'll start with a portion of the HotpotQA dataset.

HotpotQA is a question/answering dataset featuring natural questions, with correct answers. It was collected by a team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal.

Let's take a look at the demo dataset "hotpot-300".

First, we get or create it. This will add it to our default Opik project:

In [26]:
from opik_optimizer.demo import get_or_create_dataset
from opik_optimizer.demo.cache import get_litellm_cache


opik_dataset = get_or_create_dataset("hotpot-300")
get_litellm_cache("test")

Let's look at a specific row from the dataset:

In [8]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0195d400-517b-7f17-b746-ff3d084463ac',
 'question': 'Were both drinks, the Smoking Bishop and the Mickey Slim, popular in different countries?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [9]:
rows[2]

{'id': '0195d400-5179-7259-8104-1f0a55a13ac2',
 'question': 'Woody Wuthrie wrote the song "Do Re Mi" which was about migrants and their experiences when they arrive in California and are greeted with severe storms that came in waves in what years?',
 'answer': '1934, 1936, and 1939–1940'}

As you can see, the answers can be a little messy. We'll need a good metric to able to determine whether a prompt is good or not. But let's start simple.

We'll use the `LevenshteinRatio` metric from Opik.

It works like this:

In [10]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio()

metric.score("Hello", "Hello")

OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01968c66-c5d3-7e85-b7d6-3911e2e60dc8&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

Here we can see that value is 1.0. That meens that there is no difference between "Hello" and "Hello". That is as close as you can get!

As the difference between the two strings get further away, the value gets smaller, approaching 0.0:

In [11]:
metric.score("Hello", "Hello!")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

In [12]:
metric.score("one", "six")

ScoreResult(name='levenshtein_ratio_metric', value=0.0, reason=None, metadata=None, scoring_failed=False)

As you can see, slightly different have values close to 1, and no letters in common give a zero.

To find out more about the metric, see [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance).

Ok, we have a dataset and a metric, now we are ready to construct an optimizer.

We can use an OpenAI model name, or more generally, a LiteLLM model name. Just make sure you have your model API key set as we did above.

We'll start with the `FewShotBayesianOptimizer`:

In [21]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
    OptimizationConfig,
    MetricConfig,
    from_llm_response_text,
    from_dataset_field,
    PromptTaskConfig,
)

optimizer = FewShotBayesianOptimizer(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

Now, we need a prompt to optimize. Given the examples above, let's try something like:

In [22]:
initial_prompt = "Provide an answer to the question"

In [23]:
initial_prompt_no_examples = [
    {"role": "system", "content": initial_prompt},
    {"role": "user", "content": "{{question}}"},
]

In [24]:
optimization_config = OptimizationConfig(
    dataset=opik_dataset,
    objective=MetricConfig(
        metric=LevenshteinRatio(),
        inputs={
            "output": from_llm_response_text(),
            "reference": from_dataset_field(name="answer"),
        },
    ),
    task=PromptTaskConfig(
        instruction_prompt=initial_prompt,
        input_dataset_fields=["question"],
        output_dataset_field="answer",
        use_chat_prompt=True,
    ),
)

In [27]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric_config=optimization_config.objective,
    prompt=initial_prompt_no_examples,
)
score


Evaluation: 100%|██████████| 300/300 [01:03<00:00,  4.74it/s]


╭─ hotpot-300 (300 samples) ─────────────╮
│                                        │
│ Total time:        00:01:03            │
│ Number of samples: 300                 │
│                                        │
│ levenshtein_ratio_metric: 0.1453 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=981461;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01968c76-f28e-7720-ac8c-aa2edcb15b18&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.14533818431262188

The score 0.15 is pretty low. Let's see if we can optimize it!

The FewShotBayesianOptimizer can, fairly quickly, create better prompts.

Let's try it out. It takes exactly the same parameters as evaluate_prompt(), but will run for a minute or so.

In [28]:
result = optimizer.optimize_prompt(optimization_config, n_trials=3)

[I 2025-05-01 15:30:53,011] A new study created in memory with name: no-name-091bce25-dd58-4c57-8ee3-bb3781d5887d

Evaluation: 100%|██████████| 300/300 [00:39<00:00,  7.50it/s]


╭─ hotpot-300 (300 samples) ─────────────╮
│                                        │
│ Total time:        00:00:40            │
│ Number of samples: 300                 │
│                                        │
│ levenshtein_ratio_metric: 0.5314 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=26225;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01968c78-ddd8-7f5a-bf09-24dd3ba4067e&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

[I 2025-05-01 15:31:37,114] Trial 0 finished with value: 0.5314112618205721 and parameters: {'n_examples': 8, 'example_0': 251, 'example_1': 15, 'example_2': 164, 'example_3': 11, 'example_4': 118, 'example_5': 141, 'example_6': 118, 'example_7': 157}. Best is trial 0 with value: 0.5314112618205721.

Evaluation: 100%|██████████| 300/300 [00:38<00:00,  7.70it/s]


...

[I 2025-05-01 15:32:19,474] Trial 1 finished with value: 0.5372766641460849 and parameters: {'n_examples': 8, 'example_0': 294, 'example_1': 85, 'example_2': 232, 'example_3': 171, 'example_4': 104, 'example_5': 40, 'example_6': 57, 'example_7': 282}. Best is trial 1 with value: 0.5372766641460849.

Evaluation: 100%|██████████| 300/300 [00:41<00:00,  7.26it/s]


╭─ hotpot-300 (300 samples) ─────────────╮
│                                        │
│ Total time:        00:00:41            │
│ Number of samples: 300                 │
│                                        │
│ levenshtein_ratio_metric: 0.5084 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=772246;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01968c7a-3237-7785-95ad-f5171eaf3b17&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

[I 2025-05-01 15:33:04,198] Trial 2 finished with value: 0.5083562144763062 and parameters: {'n_examples': 8, 'example_0': 82, 'example_1': 180, 'example_2': 102, 'example_3': 155, 'example_4': 251, 'example_5': 236, 'example_6': 198, 'example_7': 122}. Best is trial 1 with value: 0.5372766641460849.


In [30]:
print(result)

Optimizer Results:
    Best prompt: [{'role': 'system', 'content': 'Provide an answer to the question\n\nYou are an intelligent assistant that learns from few-shot examples provided earlier in the conversation. Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies, using them as a guide'}, {'role': 'user', 'content': '\n{\n  "question": "Long Lake is located in which town?"\n}\n'}, {'role': 'assistant', 'content': 'Harrison'}, {'role': 'user', 'content': '\n{\n  "question": "In which region did the settlers have conflict with the Mexican government where it escalated to a rebellion led by John Dunn Hunter?"\n}\n'}, {'role': 'assistant', 'content': 'Texas'}, {'role': 'user', 'content': '\n{\n  "question": "What single by an Australian rock band was featured in an Australian paranormal television program which premiered on 9 July 2015, on ABC1?"\n}\n'}, {'role': 'assistant', 'content': 'Breakaway'}, {'role': 'user', 'content': '\n{\n  "quest

Although there is some randomness, you probably got a better prompt. My result is about 5 times better.

This is just the initial framework for optimizers for Opik!

Please see the [Opik Optimizer repo](https://github.com/comet-ml/opik/tree/main/sdks/opik_optimizer) for additional examples using different algorithms.